# CARTOframes 테스트  
  
- 무료 계정에서 CARTOframes의 주요 기능을 테스트해봄  
- 2020.03.17~18
  
- 배경지도  
- Point 시각화  
- Polygon (Choropleth)   
- 서비스 영역 (Isolines)

# Setup

## 데이터 처리용 라이브러리 import

In [6]:
# Data manipulation
import pandas as pd
import numpy as np

## 공간데이터 처리용 라이브러리 import

In [7]:
import geopandas as gpd
from geopandas import GeoDataFrame, points_from_xy
import fiona #공간데이터를 딕셔너리 형태 등으로 접근할 수 있는 라이브러리 
#from shapely.geometry import Polygon
#from shapely.geometry import Point
import pyproj
from fiona.crs import from_epsg

## 시각화용 라이브러리 import

In [8]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.axes_grid1 import make_axes_locatable
%matplotlib inline
mpl.rc('font', family='NanumGothic') #한글 폰트 적용시
plt.rcParams["figure.figsize"] = (20,10) #차트 사이즈

## 공간시각화용 라이브러리 import

In [9]:
#import json
#import rtree
#import folium

#import geoplot as gplt #conda install -c conda-forge geoplot
#import geoplot.crs as gcrs
#import mapclassify as mc
#import contextily as ctx

failed to import boto3, continuing.


## CARTOframes 패키지

In [11]:
from cartoframes.auth import set_default_credentials #, set_default_context
set_default_credentials('creds.json')

In [142]:
from cartoframes import * #read_carto
from cartoframes.viz import *  #Map, Layer, size_continuous_style, basemaps, basic_style, popup_element
#from cartoframes.data.services import Isolines 무료 계정 미지원
# from cartoframes.data.services import Geocoding 무료 계정 미지원-한국 주소체계 미지원?! 


# Data import

In [13]:
# 공공데이터포털의 전국도시공원표준데이터 로딩
df_park1 = pd.read_csv('./data_org/전국도시공원표준데이터.csv', \
                          index_col=None, header=0, \
                          names=['man_no', 'name', 'type', 'roadaddr', 'addr', 'lat', 'lon', 'area', 'fac1', 'fac2', 'fac3', 'fac4', 'fac5', 'date1', 'org_manage', 'tel', 'date_update', 'org_cd', 'org_info', 'b'], \
                          usecols=['name', 'type', 'lat', 'lon', 'area', 'date1', 'org_manage'], \
                          dtype={'name':object, 'type':object, 'lat':float, 'lon':float, 'area':object, 'date1':object, 'org_manage':object}, \
                          encoding="EUC-KR", delimiter=','    )
df_park1.info()
df_park1.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18867 entries, 0 to 18866
Data columns (total 7 columns):
name          18867 non-null object
type          18867 non-null object
lat           18867 non-null float64
lon           18867 non-null float64
area          18867 non-null object
date1         16053 non-null object
org_manage    18256 non-null object
dtypes: float64(2), object(5)
memory usage: 1.0+ MB


,name,type,lat,lon,area,date1,org_manage
0,화천 어린이공원1,어린이공원,38.105568,127.707075,1048,1985-06-05,강원도 화천군청
1,화천 어린이공원3,어린이공원,38.108973,127.708868,1760,1985-06-05,강원도 화천군청
2,화천 어린이공원4,어린이공원,38.114549,127.702736,1511,1985-06-05,강원도 화천군청
3,화천 어린이공원8,어린이공원,38.106931,127.706165,752,2010-08-27,강원도 화천군청
4,사내 어린이공원,어린이공원,38.069006,127.523321,1385,1998-12-14,강원도 화천군청


# Data processing

In [14]:
# '서울' 지역 데이터만 추출
df_park2 = df_park1[ df_park1['org_manage'].str.contains('서울')==True ]
df_park2.info()
df_park2.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1532 entries, 15 to 18767
Data columns (total 7 columns):
name          1532 non-null object
type          1532 non-null object
lat           1532 non-null float64
lon           1532 non-null float64
area          1532 non-null object
date1         1024 non-null object
org_manage    1532 non-null object
dtypes: float64(2), object(5)
memory usage: 95.8+ KB


,name,type,lat,lon,area,date1,org_manage
15,창신공원,어린이공원,37.578276,127.012931,710,2013-10-10,서울특별시 종로구청
16,당고개공원,어린이공원,37.575762,127.012382,860,2013-10-10,서울특별시 종로구청
17,무악공원,어린이공원,37.577525,126.960547,5136,2015-09-13,서울특별시 종로구청
18,창덕공원,근린공원,37.580274,126.991785,491328,NaN,서울특별시 종로구청
19,청진공원,근린공원,37.571796,126.979991,3341,NaN,서울특별시 종로구청


In [15]:
# 좌표 관련 전처리
#df_park3 = df_park2[['name', 'type', 'lat', 'lon', 'area', 'date1']].dropna()
df_park3 = df_park2[df_park2['lon'].notna()]
df_park3 = df_park3[df_park3['lat'].notna()]
df_park3 = df_park3[ (df_park3['lon'] >= 125)  & (df_park3['lon'] <= 128)]   
df_park3 = df_park3[(df_park3['lat'] >= 37.4)  & (df_park3['lat'] <= 38)]   
df_park3.info()
df_park3.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1530 entries, 15 to 18767
Data columns (total 7 columns):
name          1530 non-null object
type          1530 non-null object
lat           1530 non-null float64
lon           1530 non-null float64
area          1530 non-null object
date1         1022 non-null object
org_manage    1530 non-null object
dtypes: float64(2), object(5)
memory usage: 95.6+ KB


,name,type,lat,lon,area,date1,org_manage
15,창신공원,어린이공원,37.578276,127.012931,710,2013-10-10,서울특별시 종로구청
16,당고개공원,어린이공원,37.575762,127.012382,860,2013-10-10,서울특별시 종로구청
17,무악공원,어린이공원,37.577525,126.960547,5136,2015-09-13,서울특별시 종로구청
18,창덕공원,근린공원,37.580274,126.991785,491328,NaN,서울특별시 종로구청
19,청진공원,근린공원,37.571796,126.979991,3341,NaN,서울특별시 종로구청


In [16]:
# area 관련 전처리
df_park4 = df_park3[df_park3['area'].notna()]
df_park4['area'] = pd.to_numeric(df_park4['area'])
df_park4 = df_park4[ (df_park4['area'] >= 0)  & (df_park4['area'] <= 9999999999999)]   
df_park4.info()
df_park4.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1530 entries, 15 to 18767
Data columns (total 7 columns):
name          1530 non-null object
type          1530 non-null object
lat           1530 non-null float64
lon           1530 non-null float64
area          1530 non-null float64
date1         1022 non-null object
org_manage    1530 non-null object
dtypes: float64(3), object(4)
memory usage: 95.6+ KB


,name,type,lat,lon,area,date1,org_manage
15,창신공원,어린이공원,37.578276,127.012931,710.0,2013-10-10,서울특별시 종로구청
16,당고개공원,어린이공원,37.575762,127.012382,860.0,2013-10-10,서울특별시 종로구청
17,무악공원,어린이공원,37.577525,126.960547,5136.0,2015-09-13,서울특별시 종로구청
18,창덕공원,근린공원,37.580274,126.991785,491328.0,NaN,서울특별시 종로구청
19,청진공원,근린공원,37.571796,126.979991,3341.0,NaN,서울특별시 종로구청


In [17]:
gdf_park = gpd.GeoDataFrame(df_park4, geometry=points_from_xy(df_park4['lon'], df_park4['lat']))
gdf_park.head(5)

,name,type,lat,lon,area,date1,org_manage,geometry
15,창신공원,어린이공원,37.578276,127.012931,710.0,2013-10-10,서울특별시 종로구청,POINT (127.01293 37.57828)
16,당고개공원,어린이공원,37.575762,127.012382,860.0,2013-10-10,서울특별시 종로구청,POINT (127.01238 37.57576)
17,무악공원,어린이공원,37.577525,126.960547,5136.0,2015-09-13,서울특별시 종로구청,POINT (126.96055 37.57753)
18,창덕공원,근린공원,37.580274,126.991785,491328.0,NaN,서울특별시 종로구청,POINT (126.99179 37.58027)
19,청진공원,근린공원,37.571796,126.979991,3341.0,NaN,서울특별시 종로구청,POINT (126.97999 37.57180)


# 공간 시각화

In [12]:
Map(Layer(gdf_park))

## 지리적 버블맵

In [13]:
Map(Layer(gdf_park, size_continuous_style('area', size_range=[5,30],color='green', stroke_color='white', opacity=0.7), \
          title='도시공원 면적' ))

## CARTO의 다른 베이스맵

In [135]:
Map(Layer(gdf_park, size_continuous_style('area', size_range=[5,30],color='green', stroke_color='white', opacity=0.7), \
          title='도시공원 면적' ), basemaps.darkmatter)

## Mapbox Vector Tile 베이스맵

In [18]:
Map(Layer(gdf_park), basemap = {
    'style': 'mapbox://styles/mapbox/streets-v9',
    'token': 'pk.eyJ1IjoidWJlcmRhdGEiLCJhIjoiY2pudzRtaWloMDAzcTN2bzN1aXdxZHB5bSJ9.2bkj3IiRC8wj3jLThvDGdA'
})

## Mapbox XYZ Tile 베이스맵

In [2]:
import os
from mapboxgl.viz import RasterTilesViz

# Set Mapbox Acces Token; Must be a public token, starting with `pk`
token = os.getenv('pk.eyJ1IjoidWJlcmRhdGEiLCJhIjoiY2pudzRtaWloMDAzcTN2bzN1aXdxZHB5bSJ9.2bkj3IiRC8wj3jLThvDGdA')

In [3]:
#tiles_url = 'https://a.tile.openstreetmap.org/{z}/{x}/{y}.png'
tiles_url = 'http://xdworld.vworld.kr:8080/2d/Base/201908/{z}/{x}/{y}.png'
style = {'version': 8, 'sources': {}, 'layers': []}
tiles_center = [126.925, 37.523]
viz = RasterTilesViz(tiles_url, access_token=token, tiles_size=256, center=tiles_center, height='500px',  zoom=15, 
                     style=style)
viz.show()

C:\Users\gu_dhl\Anaconda3\lib\site-packages\IPython\core\display.py:701: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


# CARTO Query    
    
#### 도서관과 도시공원을 CARTO TABLE로 로딩한 후   
#### PostGIS 공간연산을 적용하여  
#### 도시공원과 가까운 도서관을 찾아보자

In [87]:
# 공공데이터포털의 전국도서관표준데이터 로딩
df_library1 = pd.read_csv('./data_org/전국도서관표준데이터.csv', \
                          index_col=None, header=0, \
                          names=['name', 'sido', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'lat', 'lon', 'z', 'aa', 'ab', 'ac' ], \
                          usecols=['name', 'sido', 'lat', 'lon'], \
                          dtype={'name':object, 'sido':object, 'lat':float, 'lon':float}, \
                          encoding="EUC-KR", delimiter=','    )
df_library1.info()
df_library1.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3193 entries, 0 to 3192
Data columns (total 4 columns):
name    3193 non-null object
sido    3193 non-null object
lat     3068 non-null float64
lon     3080 non-null float64
dtypes: float64(2), object(2)
memory usage: 99.9+ KB


,name,sido,lat,lon
0,옥과공공도서관,전라남도,35.274540,127.13557
1,도봉어린이문화정보도서관(디지털자료실),서울특별시,37.659007,127.04947
2,"도봉어린이문화정보센터(자료열람실, 잉글리시 아일랜드)",서울특별시,37.659007,127.04947
3,학마을도서관 (종합자료실),서울특별시,37.662108,127.02782
4,학마을도서관 (어린이 자료실),서울특별시,37.662110,127.02782


In [174]:
# 도서관 좌표 관련 전처리
df_library1 = df_library1[df_library1['lon'].notna()]
df_library1 = df_library1[df_library1['lat'].notna()]
df_library1 = df_library1[ (df_library1['lon'] >= 125)  & (df_library1['lon'] <= 132)]   
df_library1 = df_library1[(df_library1['lat'] >= 36.5)  & (df_library1['lat'] <= 40)]   
df_library1.info()
df_library1.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1380 entries, 1 to 3192
Data columns (total 5 columns):
name        1380 non-null object
sido        1380 non-null object
lat         1380 non-null float64
lon         1380 non-null float64
geometry    1380 non-null geometry
dtypes: float64(2), geometry(1), object(2)
memory usage: 64.7+ KB


,name,sido,lat,lon,geometry
1,도봉어린이문화정보도서관(디지털자료실),서울특별시,37.659007,127.04947,POINT (127.04947 37.65901)
2,"도봉어린이문화정보센터(자료열람실, 잉글리시 아일랜드)",서울특별시,37.659007,127.04947,POINT (127.04947 37.65901)
3,학마을도서관 (종합자료실),서울특별시,37.662108,127.02782,POINT (127.02782 37.66211)
4,학마을도서관 (어린이 자료실),서울특별시,37.662110,127.02782,POINT (127.02782 37.66211)
5,방학동영유아플라자,서울특별시,37.663230,127.03033,POINT (127.03033 37.66323)


In [175]:
# 도서관 공간데이터화
gdf_library= gpd.GeoDataFrame(df_library1, geometry=points_from_xy(df_library1['lon'], df_library1['lat']))
gdf_library.head(5)

,name,sido,lat,lon,geometry
1,도봉어린이문화정보도서관(디지털자료실),서울특별시,37.659007,127.04947,POINT (127.04947 37.65901)
2,"도봉어린이문화정보센터(자료열람실, 잉글리시 아일랜드)",서울특별시,37.659007,127.04947,POINT (127.04947 37.65901)
3,학마을도서관 (종합자료실),서울특별시,37.662108,127.02782,POINT (127.02782 37.66211)
4,학마을도서관 (어린이 자료실),서울특별시,37.662110,127.02782,POINT (127.02782 37.66211)
5,방학동영유아플라자,서울특별시,37.663230,127.03033,POINT (127.03033 37.66323)


## CARTO TABLE로 저장

In [176]:
# 도서관 공간데이터를  carto table로 저장
cartoframes.to_carto(gdf_library, 'library_korea', if_exists='replace', geom_col='geometry')

Success! Data uploaded to table "library_korea" correctly


In [93]:
# 도시공원 공간데이터를  carto table로 저장
cartoframes.to_carto(gdf_park, 'park_seoul', if_exists='replace', geom_col='geometry')

Success! Data uploaded to table "park_seoul" correctly


## SQL where 조건 검색을 이용하여 서울시 도서관만 선택

In [177]:
gdf_lib = read_carto("SELECT * FROM library_korea WHERE sido IN ('서울특별시')")
gdf_lib.info()
gdf_lib.head()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 504 entries, 0 to 503
Data columns (total 6 columns):
cartodb_id    504 non-null int64
the_geom      504 non-null geometry
name          504 non-null object
sido          504 non-null object
lat           504 non-null float64
lon           504 non-null float64
dtypes: float64(2), geometry(1), int64(1), object(2)
memory usage: 23.8+ KB


,cartodb_id,the_geom,name,sido,lat,lon
0,1,POINT (127.04947 37.65901),도봉어린이문화정보도서관(디지털자료실),서울특별시,37.659007,127.04947
1,2,POINT (127.04947 37.65901),"도봉어린이문화정보센터(자료열람실, 잉글리시 아일랜드)",서울특별시,37.659007,127.04947
2,3,POINT (127.02782 37.66211),학마을도서관 (종합자료실),서울특별시,37.662108,127.02782
3,4,POINT (127.02782 37.66211),학마을도서관 (어린이 자료실),서울특별시,37.662110,127.02782
4,5,POINT (127.03033 37.66323),방학동영유아플라자,서울특별시,37.663230,127.03033


## CARTO(PostGIS) 공간연산

In [178]:
gdf_lib2 = read_carto("SELECT l.* FROM library_korea as l, park_seoul as p WHERE ST_Dwithin(st_transform(l.the_geom, 3857), st_transform(p.the_geom, 3857), 200)" )  #60분 4km시, 분당 약 67m, 3분 200m
gdf_lib2.info()
gdf_lib2.head()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 191 entries, 0 to 190
Data columns (total 6 columns):
cartodb_id    191 non-null int64
the_geom      191 non-null geometry
name          191 non-null object
sido          191 non-null object
lat           191 non-null float64
lon           191 non-null float64
dtypes: float64(2), geometry(1), int64(1), object(2)
memory usage: 9.1+ KB


,cartodb_id,the_geom,name,sido,lat,lon
0,1208,POINT (127.01697 37.57263),아름꿈도서관,서울특별시,37.572626,127.016971
1,822,POINT (127.09430 37.50730),소나무언덕2호,서울특별시,37.507300,127.094300
2,818,POINT (127.13076 37.48103),송파글마루도서관,서울특별시,37.481029,127.130755
3,820,POINT (127.14695 37.49355),거마도서관,서울특별시,37.493545,127.146948
4,823,POINT (127.15662 37.49233),소나무언덕3호,서울특별시,37.492325,127.156619


In [180]:
Map([
    Layer(gdf_park, basic_style(color='green', size=10, opacity=0.5), default_legend=True, 
          popup_hover=[popup_element('name', title='') ], title='도시공원'),
    Layer(gdf_lib, basic_style(color='blue', size=10, opacity=0.5), 
          title='도서관', popup_click=[popup_element('name', title='') ]),
    Layer(gdf_lib2, basic_style(color='red', size=15, opacity=0.9), title='도시공원 인접 도서관')  
],
    basemap = {'style': 'mapbox://styles/mapbox/satellite-streets-v9', # 'mapbox://styles/mapbox/streets-v9',
               'token': 'pk.eyJ1IjoidWJlcmRhdGEiLCJhIjoiY2pudzRtaWloMDAzcTN2bzN1aXdxZHB5bSJ9.2bkj3IiRC8wj3jLThvDGdA' } )

## 시계열 애니메이션

In [190]:
gdf_park2 = gdf_park[gdf_park['date1'].notna()]
gdf_park2['date1'] =  pd.to_datetime(gdf_park2['date1'])
Map(Layer(gdf_park2, animation_style('date1'), default_widget=True))

C:\Users\gu_dhl\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## 단계구분도(Choropleth Map) 시각화   
### 서울시 구별 도시공원 수

In [193]:
# 전국 시군구 행정구역 공간데이터 로딩
gdf_sigungu = gpd.read_file('./data_org/SIG_201905/TL_SCCO_SIG.shp', encoding='euc-kr')  #euc-kr, euckr, utf-8, cp949 중 선택
gdf_sigungu.crs 

Failed to auto identify EPSG: 7


<Projected CRS: PROJCS["PCS_ITRF2000_TM",GEOGCS["GCS_ITRF_2000",DA ...>
Name: PCS_ITRF2000_TM
Axis Info [cartesian]:
- [east]: Easting (metre)
- [north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unnamed
- method: Transverse Mercator
Datum: International Terrestrial Reference Frame 2000
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [198]:
# 좌표변환
gdf_sigungu.crs = "epsg:5179"
gdf_sigungu4326 = gdf_sigungu.to_crs(epsg=4326)

In [199]:
gdf_sigungu4326.head()

,SIG_CD,SIG_ENG_NM,SIG_KOR_NM,geometry
0,11110,Jongno-gu,종로구,"POLYGON ((127.00864 37.58047, 127.00871 37.580..."
1,11140,Jung-gu,중구,"POLYGON ((127.02314 37.57196, 127.02336 37.571..."
2,11170,Yongsan-gu,용산구,"POLYGON ((126.96918 37.55566, 126.96917 37.554..."
3,11200,Seongdong-gu,성동구,"POLYGON ((127.04341 37.57234, 127.04524 37.571..."
4,11215,Gwangjin-gu,광진구,"POLYGON ((127.10166 37.57240, 127.10224 37.572..."


In [200]:
#서울시 구 행정구역만 추출
gdf_sigungu4326['SIG_CD'] = gdf_sigungu4326['SIG_CD'].astype(str)
gdf_sigungu4326['DO'] = gdf_sigungu4326['SIG_CD'].str.slice(start=0, stop=2) 
gdf_sigungu4326_sl = gdf_sigungu4326[gdf_sigungu4326['DO'].str.contains('11')==True]  
gdf_sigungu4326_sl.info()
gdf_sigungu4326_sl.head()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 25 entries, 0 to 24
Data columns (total 5 columns):
SIG_CD        25 non-null object
SIG_ENG_NM    25 non-null object
SIG_KOR_NM    25 non-null object
geometry      25 non-null geometry
DO            25 non-null object
dtypes: geometry(1), object(4)
memory usage: 1.2+ KB


,SIG_CD,SIG_ENG_NM,SIG_KOR_NM,geometry,DO
0,11110,Jongno-gu,종로구,"POLYGON ((127.00864 37.58047, 127.00871 37.580...",11
1,11140,Jung-gu,중구,"POLYGON ((127.02314 37.57196, 127.02336 37.571...",11
2,11170,Yongsan-gu,용산구,"POLYGON ((126.96918 37.55566, 126.96917 37.554...",11
3,11200,Seongdong-gu,성동구,"POLYGON ((127.04341 37.57234, 127.04524 37.571...",11
4,11215,Gwangjin-gu,광진구,"POLYGON ((127.10166 37.57240, 127.10224 37.572...",11


In [202]:
# 구 행정구역에 도시공원 포인트를 공간조인
df_gu_park = gpd.sjoin(gdf_sigungu4326_sl, gdf_park) #, how='left', op='intersects')
df_gu_park.info()
df_gu_park.head()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 1527 entries, 0 to 24
Data columns (total 13 columns):
SIG_CD         1527 non-null object
SIG_ENG_NM     1527 non-null object
SIG_KOR_NM     1527 non-null object
geometry       1527 non-null geometry
DO             1527 non-null object
index_right    1527 non-null int64
name           1527 non-null object
type           1527 non-null object
lat            1527 non-null float64
lon            1527 non-null float64
area           1527 non-null float64
date1          1019 non-null object
org_manage     1527 non-null object
dtypes: float64(3), geometry(1), int64(1), object(8)
memory usage: 167.0+ KB


C:\Users\gu_dhl\Anaconda3\lib\site-packages\geopandas\tools\sjoin.py:61: UserWarning: CRS of frames being joined does not match!(epsg:4326 != None)
  "(%s != %s)" % (left_df.crs, right_df.crs)


,SIG_CD,SIG_ENG_NM,SIG_KOR_NM,geometry,DO,index_right,name,type,lat,lon,area,date1,org_manage
0,11110,Jongno-gu,종로구,"POLYGON ((127.00864 37.58047, 127.00871 37.580...",11,45,평창공원,소공원,37.610096,126.977439,1117.6,2010-05-13,서울특별시 종로구청
0,11110,Jongno-gu,종로구,"POLYGON ((127.00864 37.58047, 127.00871 37.580...",11,36,후창공원,소공원,37.610258,126.964365,1920.0,2009-05-28,서울특별시 종로구청
0,11110,Jongno-gu,종로구,"POLYGON ((127.00864 37.58047, 127.00871 37.580...",11,22,감나무골공원,어린이공원,37.612876,126.970377,2403.0,1971-06-28,서울특별시 종로구청
0,11110,Jongno-gu,종로구,"POLYGON ((127.00864 37.58047, 127.00871 37.580...",11,21,참샘골공원,어린이공원,37.615353,126.974640,2737.0,1971-06-28,서울특별시 종로구청
0,11110,Jongno-gu,종로구,"POLYGON ((127.00864 37.58047, 127.00871 37.580...",11,19,청진공원,근린공원,37.571796,126.979991,3341.0,NaN,서울특별시 종로구청


In [203]:
# 행별로 1씩 넣은 후 구 이름으로 GROUPBY하여 구별 도시공원 포인트 갯수를 산출
df_gu_park['count'] = 1
df_gu_park = df_gu_park['count'].groupby( df_gu_park['SIG_KOR_NM'] ).sum().reset_index(name='count') 
df_gu_park.head()

,SIG_KOR_NM,count
0,강남구,1
1,강동구,74
2,강북구,45
3,강서구,139
4,관악구,78


In [204]:
#집게 속성을 구 행정구역에 속성조인
gdf_gu_park = pd.merge(gdf_sigungu4326_sl, df_gu_park, how='left', on='SIG_KOR_NM') #, right_on='gu')
gdf_gu_park.head()

,SIG_CD,SIG_ENG_NM,SIG_KOR_NM,geometry,DO,count
0,11110,Jongno-gu,종로구,"POLYGON ((127.00864 37.58047, 127.00871 37.580...",11,34.0
1,11140,Jung-gu,중구,"POLYGON ((127.02314 37.57196, 127.02336 37.571...",11,33.0
2,11170,Yongsan-gu,용산구,"POLYGON ((126.96918 37.55566, 126.96917 37.554...",11,1.0
3,11200,Seongdong-gu,성동구,"POLYGON ((127.04341 37.57234, 127.04524 37.571...",11,60.0
4,11215,Gwangjin-gu,광진구,"POLYGON ((127.10166 37.57240, 127.10224 37.572...",11,24.0


In [212]:
Map(Layer(gdf_gu_park, color_bins_style('count', palette='BluGrn', opacity=0.6), \
          title='서울시 구별 도시공원 수' ), )
#https://carto.com/carto-colors/

## 웹서비스 발행

In [14]:
tmap = Map(Layer(gdf_park, size_continuous_style('area', size_range=[5,30],color='green', stroke_color='white', opacity=0.7), \
          title='도시공원 면적' ))
tmap.publish('seoul_park_analysis', password=None, if_exists='replace')

{'id': '9cd16ce3-7954-4b46-8db2-ff8f29a0ebc6',
 'url': 'https://geodata357.carto.com/kuviz/9cd16ce3-7954-4b46-8db2-ff8f29a0ebc6',
 'name': 'seoul_park_analysis',
 'privacy': 'public'}